In [1]:
from  bs4 import BeautifulSoup as bs
import requests
import re

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}

In [3]:
url = 'https://www.imdb.com/find?q=inception&ref_=nv_sr_sm'

In [4]:
webpage = requests.get(url, headers=headers).text
soup = bs(webpage, 'lxml')

In [5]:
res=soup.find_all('div',class_='findSection')

In [6]:
res=res[0]

In [7]:
for i in res.find_all('td', class_="result_text"):
    print((i.text).strip())

Inception (2010)
Inception (2014) (Short)
Inception (2017) (TV Episode)  - Season 1 | Episode 6  - La Scene en Plus (2017) (TV Mini-Series)
Inception: The Cobol Job (2010) (Video)


In [8]:
##'a', href=re.compile(r'[/]([a-z]|[A-Z])\w+')).attrs['href']
for i in res.find_all('td', class_="result_text"):
    print((i.find('a', href=re.compile(r'[/]([a-z]|[A-Z])\w+')).attrs['href']))


/title/tt1375666/?ref_=fn_al_tt_1
/title/tt7321322/?ref_=fn_al_tt_2
/title/tt6716336/?ref_=fn_al_tt_3
/title/tt5295894/?ref_=fn_al_tt_4


In [9]:
def remove_special(text):
    x = ''

    for i in text:
        if i.isalnum():
            x = x + i
        else:
            x = x + ' '
    return x

In [10]:
def url_link(text):
    clean = re.compile('/title/')
    x = re.sub(clean, '', text)
    i=0
    res=''
    while i<len(x):
        if x[i]=='/':
            break
        else:
            res+=x[i]
        i=i+1
    return res

In [11]:
url_link('/title/tt1375666/?ref_=fn_al_tt_1')

'tt1375666'

In [12]:
def search_item(movie_name):
    if len(movie_name)==0:
        return "Write Movie Name"
    else:
        url = "https://www.imdb.com/find?q=" + movie_name + "&ref_=nv_sr_sm"
        webpage = requests.get(url, headers=headers).text
        soup = bs(webpage, 'lxml')
        scr_res = soup.find_all('div', class_='findSection')
        movies, url_movie_detail_link = [], []
        for i in scr_res[0].find_all('td', class_="result_text"):
            movies.append((i.text).strip())
            url_movie_detail_link.append(url_link(i.find('a', href=re.compile(r'[/]([a-z]|[A-Z])\w+')).attrs['href']))

        return movies, url_movie_detail_link

In [13]:
search_item('inception')

(['Inception (2010)',
  'Inception (2014) (Short)',
  'Inception (2017) (TV Episode)  - Season 1 | Episode 6  - La Scene en Plus (2017) (TV Mini-Series)',
  'Inception: The Cobol Job (2010) (Video)'],
 ['tt1375666', 'tt7321322', 'tt6716336', 'tt5295894'])

In [14]:
m,code=search_item('dhoom')

In [15]:
code

['tt0422091', 'tt0441048', 'tt1833673', 'tt3239712']

In [16]:
code=code[0]

In [17]:
code

'tt0422091'

In [18]:
url1 = "https://www.imdb.com/title/{}/?ref_=fn_tt_tt_1".format(code)
url2 = "https://www.imdb.com/title/{}/reviews?ref_=tt_urv".format(code)

In [19]:
url1

'https://www.imdb.com/title/tt0422091/?ref_=fn_tt_tt_1'

In [20]:
url2

'https://www.imdb.com/title/tt0422091/reviews?ref_=tt_urv'

In [21]:
def movie_img(url):
    ul=''
    webpage=requests.get(url, headers= headers).text
    soup=bs(webpage,'lxml')
    for i in soup.find_all('div',class_='poster'):
        ul = (i.find('img')).get('src')
    return ul

In [22]:
movie_img(url1)

'https://m.media-amazon.com/images/M/MV5BOTk3MDNhODEtMWYyMC00NmVjLTg3NzgtNjI1MzA4ZmVhMjE2XkEyXkFqcGdeQXVyNTkzNDQ4ODc@._V1_UY268_CR9,0,182,268_AL__QL50.jpg'

In [23]:
def movie_plot(url):
    plot=''
    webpage=requests.get(url, headers= headers).text
    soup=bs(webpage,'lxml')
    for i in soup.find_all('div',class_='summary_text'):
        if (((i.text).strip()=='Add a Plot »') or (len((i.text).strip())<15)):
            plot = 'Not Available'
        else:
            plot = (i.text).strip()
    return plot

In [24]:
movie_plot(url1)

"A mysterious gang of bikers is on a robbing spree. ACP Jai gets Ali, a mechanic, to assist him in the case. With the clock ticking, it's up to them to nab the thieves red-handed."

In [25]:
def movie_details(url):
    webpage = requests.get(url, headers=headers).text
    soup = bs(webpage, 'lxml')
    movie_name = ''
    l = []
    movie_rating = ''
    for i in soup.find_all('h1', class_=""):
        movie_name = (remove_special(i.text).strip()).strip()
    for i in soup.find_all('div', class_='ratingValue'):
        movie_rating = (i.text).strip()

    for i in soup.find_all('div', class_="credit_summary_item"):
        l.append(remove_special((i.text).strip()).strip())
    s = []
    f = []
    val = 0
    while val < len(l):
        for i in l[val].split(' '):
            if i == 'See':
                break
            else:
                s.append(i.strip())
                x = ' '.join(s)
        f.append(x.strip())
        s.clear()

        val = val + 1

    return movie_name, movie_rating, f

In [26]:
movie_details(url1)  

('Dhoom  2004',
 '6.7/10',
 ['Director  Sanjay Gadhvi',
  'Writers  Vijay Krishna Acharya  dialogue   Vijay Krishna Acharya  screenplay',
  'Stars  Abhishek Bachchan  John Abraham  Uday Chopra'])

In [27]:
def site_review(u):
    page = requests.get(u, headers=headers).text

    soup = bs(page, 'lxml')
    pg = soup.find_all('div', class_='lister-item-content')

    url_user_name, url_title, url_rating, url_reviews = [], [], [], []  ## content

    for i in pg:
        try:
            url_title.append((i.find('a', class_="title").text).strip())
        except:
            url_title.append('not available')
        try:
            url_rating.append((i.find('span', class_="rating-other-user-rating").text).strip())
        except:
            url_rating.append('not available')
        try:
            url_user_name.append((i.find('span', class_="display-name-link").text).strip())
        except:
            url_user_name.append('not available')
        try:
            url_reviews.append((i.find('div', class_="text show-more__control").text).strip())
        except:
            url_reviews.append('not available')
    return url_user_name, url_title, url_rating, url_reviews

In [28]:
user_name,review_title,review_rating,review = site_review(url2)

In [30]:
user_name

['MR_Heraclius',
 'AishFan',
 'darashukoh',
 'AvinashPatalay',
 'Vishal_s_kumar',
 'blooming_jasmine',
 'Angelus2',
 'deavenger',
 'DVD_Connoisseur',
 'tharukshan12',
 'paulclaassen',
 'morrison-dylan-fan',
 'Manuja',
 'raju2k84',
 'freddiejason',
 'tarjeiol-1',
 'DICK STEEL',
 'harvinderbanselis',
 'macleod_lord',
 'ishel',
 'soumyadeep980',
 'labeebster',
 'suri_ramneek-2',
 'sharath-04377',
 'm_sebastian']

In [31]:
review_title

['Dhoom',
 'Adrenaline rush - welcome to Bollywood!',
 "Don't think and you might be entertained",
 'Fast and Furious in Bollywood',
 'Zip!! Zap!! Zooooommmmmm!!!!!',
 "Summary of The Fast and the Furious, Ocean's Eleven and some more action flicks + some Hindi movie masala",
 "There's no turning round! Dhoom Machale!!!!!!",
 "Surprising isn't it.",
 'Glossy, fast-moving and amusing action film',
 'John steals the show...............',
 'An absolute pleasure to watch.',
 'The Dhoom series:Part 1.',
 '"Dhoom" is the genesis of the new "Indian Action Film"!',
 'Dhoom is just for kids',
 'Good action, good looking people, BAD acting',
 "So bad it's funny",
 'A Nutshell Review: (DVD) Dhoom (2004)',
 'Stupid Vs Style',
 'Very entertaining',
 'Hot, fast-paced and whimsical',
 'Not Recommended 👎',
 'An enjoyable movie which could have been better with a better cast',
 'Time pass entertainment',
 'Cool stuff',
 "It's perfect for what it is!"]

In [33]:
review_rating

['7/10',
 '8/10',
 'not available',
 'not available',
 'not available',
 'not available',
 '8/10',
 '7/10',
 'not available',
 '8/10',
 '6/10',
 '8/10',
 '8/10',
 'not available',
 '10/10',
 '2/10',
 '8/10',
 '8/10',
 '10/10',
 '8/10',
 '1/10',
 '8/10',
 '5/10',
 '9/10',
 '9/10']

In [34]:
review_rating,review

(['7/10',
  '8/10',
  'not available',
  'not available',
  'not available',
  'not available',
  '8/10',
  '7/10',
  'not available',
  '8/10',
  '6/10',
  '8/10',
  '8/10',
  'not available',
  '10/10',
  '2/10',
  '8/10',
  '8/10',
  '10/10',
  '8/10',
  '1/10',
  '8/10',
  '5/10',
  '9/10',
  '9/10'],
 ['This is the first Bollywood action movie I have seen. It was outrageous! It reminded me of the first Fast and the Furious and The Transporter, which is probably the best you could hope for in a Bollywood action movie. There was also a fight scene on top of the trailer of an 18-wheeler extremely reminiscent of The Matrix Reloaded, which is not a good thing considering this movie is not set in a virtual reality. But its all in good fun. With hot actors, great music, and lots of comedy mixed in, this is great for the not-so-serious moviegoer.',
  'High speed chases, extraordinary camera-work, wizardry technical effects, and sensuous bombshells - the thrilling "sub-genre" of adrenaline